In [1]:
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Import data

In [2]:
df = pd.read_csv("data/kompas.csv")
df

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."
...,...
2003,Tersangka Peledakan Granat di Bulungan Ditangk...
2004,Soal Operasi Pasar Murni Beras Pedagang Yaki...
2005,Penjualan Indomobil Langgar Prosedur\n\nSelain...
2006,Belum Jelas Motif Peledakan Granat di Blok M\...


# oke mari kita praktek

## Extract TFIDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo)

In [5]:
tfidf_matrix = tfidf.fit_transform(df.teks)

C:\Users\IKIN\Miniconda3\envs\jcopml\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# TFIDF Similarity -> Document Similarity

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim


array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [8]:
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]], dtype=int64)

In [9]:
# oke kita buktikan yaa gays ya
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [10]:
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [11]:
# dokumen paling gamirip
df.teks[932][:200]

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

# Keyword Extraction

In [12]:
vocab = tfidf.get_feature_names()
vocab[-10:]

['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [13]:
# vocab ke 0 tfidfnya adalah 0.02115058
tfidf_matrix[0].toarray()


array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [14]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274212, 365469, 365468, ..., 386379, 436652, 169219], dtype=int64)

In [15]:
vocab[169219]

'ginandjar'

In [16]:
vocab[436633]

'putus indonesia'

In [17]:
[vocab[idx]for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

In [18]:
# bungkus menggunakan fungsi supaya enak dipakai
def extract_keywords_tfidf(doc, tfidf, topk=10):
    matrix = tfidf.transform(text)
    vocab = tfidf.get_feature_names()
    
    sorted_tfidf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]